<a href="https://colab.research.google.com/github/yoojonghyun/Math_Box/blob/main/scraping_plus_threeline%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W09-1_text-summerization 송부

---

In [19]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

print("1111111111")

# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from openpyxl import Workbook

import csv
import time

print("222222222")

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')        # Head-less 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

print("33333333333")

wb = Workbook()
ws = wb.active
ws.append(['keyword', 'source', 'datetime', 
           'title', 'content', 'article_url', 'short'])

print("44444444444")

# 수집할 뉴스기사 정보를 입력합니다.
QUERY = "케이뱅크"          # 필터링 키워드
START_DATE = "2022.02.22" # 필터링 일자 (작성일 기준)
END_DATE = "2022.02.22"
START_PAGE = 1            # 검색결과 저장 페이지 범위 (네이버 뉴스기사는 검색결과 중 최대 4,000페이지만 제공)
END_PAGE = 1     
                          # 기사와 댓글을 저장할 파일명
article_filename = "article_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"
# comment_filename = "comment_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"

print("55555555555")

fa = open(article_filename, "w", encoding="utf-8")
# fc = open(comment_filename, "w", encoding="utf-8")
news_count = 0


print("6666666666666")

for page in range(START_PAGE, END_PAGE+1):
    #print(page,  "번째 기시부터 최대 10개 수집중...", end="\r")
    URL = "https://search.naver.com/search.naver?&where=news&query=" + QUERY 
    URL += "&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds="
    URL += START_DATE + "&de=" + END_DATE + "&docid=&&start=" + str(page) + "&refresh_start=0"
    driver.get(URL)
    time.sleep(2)
    
    try:
        news_list = driver.find_element_by_class_name("list_news").find_elements_by_class_name("bx")
    except:
        break
    
    news_count += len(news_list)
    for news in news_list[:]:
        link_list = news.find_element_by_class_name("info_group").find_elements_by_tag_name("a")
        if len(link_list) == 1:
            continue
        article_url = link_list[1].get_attribute("href").strip()
        link_list[1].click()
        time.sleep(3)
        current_window = driver.current_window_handle
        try:
            new_window = [window for window in driver.window_handles if window != current_window][0]
            driver.switch_to.window(new_window)
        except:
            driver.switch_to.window(current_window)
            continue
        time.sleep(4)
        
        try:
            try:
                source_label = driver.find_element_by_class_name("press_logo")
            except:
                source_label = driver.find_element_by_id("pressLogo")
            source_img = source_label.find_element_by_tag_name("img")
            source = source_img.get_attribute("alt").strip()

            datetime = ""
            content = ""
            title = ""
            etc_good_count = ""
            etc_warm_count = ""
            etc_sad_count = ""
            etc_angry_count = ""
            etc_want_count = ""
            ent_good_count = ""
            ent_cheer_count = ""
            ent_congrats_count = ""
            ent_expect_count = ""
            ent_suprise_count = ""
            ent_sad_count = ""
            basic_good_count = ""
            basic_sad_count = ""
            basic_angry_count = ""
            basic_fan_count = ""
            basic_want_count = ""

            try:
                # 기타 섹션 기사
                datetime = driver.find_element_by_class_name("t11").text.strip()
                content = driver.find_element_by_class_name("_article_body_contents").text.strip().replace("\n", " ")
                title = driver.find_element_by_class_name("tts_head").text.strip()
                reaction_list = driver.find_element_by_class_name("end_btn").find_element_by_class_name("_reactionModule").find_elements_by_tag_name("a")
                etc_good_count = reaction_list[0].find_element_by_class_name("u_likeit_list_count").text.strip()
                etc_warm_count = reaction_list[1].find_element_by_class_name("u_likeit_list_count").text.strip()
                etc_sad_count = reaction_list[2].find_element_by_class_name("u_likeit_list_count").text.strip()
                etc_angry_count = reaction_list[3].find_element_by_class_name("u_likeit_list_count").text.strip()
                etc_want_count = reaction_list[4].find_element_by_class_name("u_likeit_list_count").text.strip()
            except:
                try:
                    # 연예 섹션 기사
                    datetime = driver.find_element_by_class_name("author")
                    datetime = datetime.find_element_by_tag_name("em").text.strip()
                    content = driver.find_element(By.ID, "articeBody").text.strip().replace("\n", " ")
                    title = driver.find_element_by_class_name("end_tit").text.strip()
                    reaction_list = driver.find_element_by_class_name("end_btn").find_element_by_class_name("_reactionModule").find_elements_by_tag_name("a")
                    ent_good_count = reaction_list[0].find_element_by_class_name("u_likeit_list_count").text.strip()
                    ent_cheer_count = reaction_list[1].find_element_by_class_name("u_likeit_list_count").text.strip()
                    ent_congrats_count = reaction_list[2].find_element_by_class_name("u_likeit_list_count").text.strip()
                    ent_expect_count = reaction_list[3].find_element_by_class_name("u_likeit_list_count").text.strip()
                    ent_suprise_count = reaction_list[4].find_element_by_class_name("u_likeit_list_count").text.strip()
                    ent_sad_count = reaction_list[5].find_element_by_class_name("u_likeit_list_count").text.strip()
                except:
                    # 일반 섹션 기사
                    datetime = driver.find_element_by_class_name("info").find_element_by_tag_name("span").text.replace("기사입력", "").strip()
                    content = driver.find_element_by_class_name("news_end").text.strip().replace("\n", " ")
                    title = driver.find_element_by_class_name("title").text.strip()
                    reaction_list = driver.find_element_by_class_name("news_end_btn").find_element_by_class_name("_reactionModule").find_elements_by_tag_name("a")
                    basic_good_count = reaction_list[0].find_element_by_class_name("u_likeit_list_count").text.strip()
                    basic_sad_count = reaction_list[1].find_element_by_class_name("u_likeit_list_count").text.strip()
                    basic_angry_count = reaction_list[2].find_element_by_class_name("u_likeit_list_count").text.strip()
                    basic_fan_count = reaction_list[3].find_element_by_class_name("u_likeit_list_count").text.strip()
                    basic_want_count = reaction_list[4].find_element_by_class_name("u_likeit_list_count").text.strip()

                    
                    
                    
            review_count_list = driver.find_elements_by_class_name("u_cbox_count")
            if len(review_count_list) > 0:
                review_count = review_count_list[0].text.replace(",", "")
            else:
                review_count = "0"
            #print(source + "\t" + datetime + "\t" + review_count + "\t" + 
            #      good_count + "\t" + warm_count + "\t" + sad_count + "\t" + 
            #      angry_count + "\t" + want_count + "\t" + article_url + "\t" + 
            #      title + "\t" + content)
            print(" "*100, end="\r")
            print(str(page)+"/"+str(END_PAGE)+" Page,", title + " (댓글: "+str(review_count)+"건)", end="\r")
            fa.write(source + "\t" + datetime + "\t" + review_count + "\t" + 
                     ent_good_count + "\t" + ent_cheer_count + "\t" + ent_congrats_count + "\t" + 
                     ent_expect_count + "\t" + ent_suprise_count + "\t" + ent_sad_count + "\t" + 
                     basic_good_count + "\t" + basic_sad_count + "\t" + basic_angry_count + "\t" + 
                     basic_fan_count + "\t" + basic_want_count + "\t" + 
                     etc_good_count + "\t" + etc_warm_count + "\t" + etc_sad_count + "\t" + 
                     etc_angry_count + "\t" + etc_want_count + "\t" + 
                     article_url + "\t" + 
                     title + "\t" + content + "\n")
            # *****************************************

            # 기사 요약 내용 추가
            from teanaps.handler import FileHandler
            from teanaps.nlp import Processing  
            fh = FileHandler()
            pro = Processing()
            print("article 1")
            from teanaps.text_analysis import DocumentSummarizer
            ds = DocumentSummarizer()

            print(content)

            line_list = fh.load_txt(content)
            print(line_list, "LISTLISTLIST")

            SUMMARIZER_TYPE = "textrank"
            MAX_SENTENCES = 3
            short = ds.summarize(SUMMARIZER_TYPE, MAX_SENTENCES, document=line_list)
            print(5432123456789)
            print("1111111"+short)

            ws.append([QUERY, source, datetime, title, content, article_url, short])
            wb.save("/content/drive/MyDrive/Colab Notebooks/result.xlsx")
            time.sleep(2)
            
            
            
            # 댓글 더보기 클릭
            if int(review_count) > 0:
                driver.find_element_by_class_name("u_cbox_in_view_comment").click()
                time.sleep(3)

                # 더보기 버튼 클릭
                for i in range(10):
                    try:
                        more_button_status = driver.find_element_by_class_name("u_cbox_paginate").get_attribute("style").strip()
                        if more_button_status == '':
                            driver.find_element_by_class_name("u_cbox_more_wrap").click()
                        time.sleep(1)
                    except:
                        continue

                # 답글 클릭
                comment_list = driver.find_elements_by_class_name("u_cbox_area")
                driver.execute_script('window.scrollTo(0, 0);')
                for comment in comment_list:
                    reply_count_list = comment.find_elements_by_class_name("u_cbox_btn_reply")
                    if len(reply_count_list) > 0:
                        reply_count = reply_count_list[0].text.strip()
                        if reply_count != "답글0":
                            #reply_button = comment.find_element_by_class_name("u_cbox_btn_reply")
                            #reply_button.click()
                            reply_count_list[0].click()
                time.sleep(3)


        except:
            driver.close()
            time.sleep(3)
            driver.switch_to.window(current_window)
            fa.flush()
            continue
        
        
        driver.close()
        time.sleep(3)
        driver.switch_to.window(current_window)
        fa.flush()
        
fa.close()

# 수집종료
print()
print("* 최대", news_count, "개 기사 수집이 완료되었습니다.")
print("* 수집된 기사는 아래 파일에 저장되었습니다.")
print("  - 기사본문 :", article_filename)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:464: UserWarning:

find_elements_by_* commands are deprecated. Please use find_elements() instead

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:446: UserWarning:

find_element_by_* commands are deprecated. Please use find_element() instead

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:359: UserWarning:

find_elements_by_* commands are deprecated. Please use find_elements() instead

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:341: UserWarning:

find_element_by_* commands are deprecated. Please use find_element() instead



article 1
3040 ‘젊은 큰손들’ 급증… 고객 유치 치열한 경쟁 “투자금 480억원이 1시간 반 만에 모여서 깜짝 놀랐습니다.”  삼성증권의 30억원 이상 고액 자산가 전담 컨설팅 조직 ‘SNI’의 백혜진 상무는 작년 5월 인터넷전문은행 케이뱅크의 유상증자 당시 ‘영앤리치(young & rich·젊은 부자)’의 위력을 실감했다고 한다. 삼성증권은 케이뱅크 유상증자 금액 중 480억원을 조달하기로 하고 이를 ‘The SNI’에 맡겼는데, 불과 1시간30분 만에 목표액이 채워졌다. ‘큰손’ 고객들을 관리하는 PB(프라이빗 뱅커) 센터에서는 며칠씩 걸리던 일이 순식간에 끝난 것이다. 백 상무는 “벤처 출신 거부들은 벤처 기업을 경영하며 직접 투자를 받아본 경험이 있기 때문에 투자에 대해 길게 설명할 필요가 없었다”며 “재무 담당 비서를 따로 두지 않고 대부분 직접 투자처를 정하기 때문에 의사결정 속도도 더 빠르다”고 했다.  /그래픽=양진경  지난해 글로벌 컨설팅 기업 보스턴컨설팅그룹(BCG) 조사 결과, 이런 신흥 부자가 보유한 자산인 ‘뉴머니’ 규모는 ‘올드머니’보다 2배 빠르게 성장해 2030년에는 전체 부유층 자산의 절반을 차지할 것으로 관측됐다.  이처럼 급성장하는 30~40대 영 앤 리치를 잡기 위한 금융권 경쟁이 치열하다. 영 앤 리치는 주로 주식 지분 매각 등으로 자산을 형성했기 때문에 부동산 못지않게 금융 투자를 선호한다고 한다. IPO(상장)나 스톡옵션, 가상화폐 등으로 부를 거머쥔 젊은 고액 자산가는 이미 빠르게 늘고 있다. 작년 말 미래에셋증권의 30억원 이상 보유 고객 2710명 중 30·40대는 18%(489명)로, 3년 만에 2.7배로 불어났다. NH투자증권의 30억원 이상 보유 고객도 2019년 319명에서 2021년 440명으로 2년 만에 37% 늘었는데 이들의 1인당 평균 자산은 308억원에 달했다. 젊은 ‘큰손’이 금융사의 ‘미래 수익원’으로 부상하는 이유다.  똑똑하고 까다로운 영 앤 리치 입맛  스타트업에서 출발해 ‘대

위쪽은 크롤링

In [9]:
path = ""

In [10]:
!ls

article_토스뱅크_2022.02.22_2022.02.22_1.txt  drive  sample_data


In [11]:
!git clone https://github.com/fingeredman/teanaps.git

Cloning into 'teanaps'...
remote: Enumerating objects: 1981, done.
remote: Counting objects: 100% (476/476), done.
remote: Compressing objects: 100% (446/446), done.
remote: Total 1981 (delta 287), reused 38 (delta 30), pack-reused 1505
Receiving objects: 100% (1981/1981), 63.91 MiB | 24.45 MiB/s, done.
Resolving deltas: 100% (1228/1228), done.


In [12]:
!ls

article_토스뱅크_2022.02.22_2022.02.22_1.txt  drive  sample_data  teanaps


In [ ]:
!python "teanaps/teanaps_setup.py"

     |████████████████████████████████| 24.2 MB 49.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 25.0 MB 2.2 MB/s 
     |████████████████████████████████| 127 kB 55.7 MB/s 
  Created wheel for plotly: filename=plotly-2.7.0-py3-none-any.whl size=25015296 sha256=dd31bc553175ea860c15bd79014424513d2af7574886bec52c963765b0d9f277
  Stored in directory: /root/.cache/pip/wheels/d5/ac/7b/4036185f81bd3469a68b23486e0c7660bd8f07c172e25a0873
Successfully built plotly
  Attempting 

### 1. 뉴스기사 불러오기

---

In [8]:
from teanaps.handler import FileHandler
from teanaps.nlp import Processing

fh = FileHandler()
pro = Processing()

PATH = "/content/article_토스뱅크_2022.02.22_2022.02.22_1.txt"
line_list = fh.load_txt(PATH)

ModuleNotFoundError: ignored

In [ ]:
len(line_list)

5

In [ ]:
line_list[2]

['ZDNet Korea',
 '2022.02.22. 오전 9:56',
 '0',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '0',
 '0',
 '0',
 '0',
 '0',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002248301',
 'JT저축은행, 총자산 2조원 돌파…출범 7년 만',
 '최성욱 대표 "디지털 서비스 강화" JT저축은행이 2015년 출범 이후 7년 만에 총자산 규모가 약 6배 늘어나 2조원을 돌파하면서 \'대형 저축은행\'에 한 걸음 가까워졌다.  22일 JT저축은행은 2015년 1월 SC저축은행 인수 시 약 3천369억 원이던 총자산이 지난 1월 말 기준 약 2조1천500억원으로 증가했다고 밝혔다.  출범 당시 총 수신 금액(잔액 기준)은 2천725억원서 지난 1월 말 기준 1조9천122억원으로 약 7배 증가했다.  여신 포트폴리오 균형을 위해 출범 당시 햇살론과 일반신용대출에 쏠려있던 사업 구조를 다변화했다. 2016년 기업금융팀을 신설하는 등 사업 구조 조정을 통해 지난 1월말 기준 기업과 가계대출 비중이 5.5대 4.5로 달라졌다.  JT저축은행 최성욱 대표.  JT저축은행은 디지털 역량 강화에 집중한다는 계획이다. 회사는 혁신플랫폼사업팀을 신설해 토스·카카오페이·핀크·시럽·마이뱅크 등과 제휴를 확대하고 있으며, 자동 심사 기능을 탑재한 모바일 즉시 대출 서비스를 제공 중이다.  이밖에 내부 고객 데이터를 분석하는 데이터마이닝팀을 만들었다.  최성욱 JT저축은행 대표는 "디지털 서비스의 편의성이 나날이 중요해지는 이때 서비스 개선 뿐 아니라 대내외적 리스크 관리 역량을 더욱 강화해 지속 가능한 성장을 이어갈 것"이라고 말했다.']

### 2. 뉴스기사 요약하기

---

#### 2.1. 기사 불러오기

---

In [ ]:
from teanaps.text_analysis import DocumentSummarizer

ds = DocumentSummarizer()

In [ ]:
line_list[0][-1]

'최서룡 한투증권 본부장 “오랜기간 얻은 데이터 바탕으로 더 정확한 투자 진단할 수 있고, 원앱보다 세분화된 앱이 중요” “디지털 전략의 핵심은 리스크 진단입니다.”  21일 동아일보와의 인터뷰에서 최서룡 한국투자증권 디지털플랫폼 본부장(사진)은 이같이 밝혔다. 최 본부장은 “유동성 장세가 지나며 얼마나 높은 수익률을 낼 수 있는지보다 얼마만큼 잃을 수 있는지 리스크를 계산하는 게 중요해졌다”고 했다.  최근 토스, 카카오 등 증권업에 진출하는 빅테크 기업들이 늘어나는 가운데 전통 증권사들도 디지털 역량 강화에 박차를 가하고 있다. 한국투자증권은 디지털플랫폼 본부를 두고 소수점 해외주식 거래 ‘미니스탁’, 마이데이터 ‘모이다’ 등 디지털플랫폼 구축과 운용에 적극 나서고 있다.  빅테크와 차별화된 경쟁력을 가지려면 기존 증권사만의 강점을 활용해야 한다는 게 최 본부장의 생각이다. 기존 증권사의 가장 큰 자산은 리스크 관리 노하우다. 그는 “오랜 기간 금융투자업계에서 얻은 데이터를 바탕으로 금융상품 투자 손실 가능성 및 비용 등을 기존 빅테크보다 정확하게 진단할 수 있다”고 했다.  빅테크의 장점으로 언급되는 ‘원앱’ 전략에 대해서도 무조건 따라 할 건 아니라고 보고 있다. 토스는 토스뱅크, 증권, 송금 서비스 등을 한데 모아놓은 ‘원앱’ 전략을 내놓았고, 간편함으로 시장의 호평을 받았다. 최 본부장은 “금융상품 투자부터 연금 투자, 선물 등 이미 많은 기능을 갖고 있어 이를 앱 하나로 모으는 것은 불가능하다. 오히려 기능을 세분화하고 기능에 맞게 앱을 추천하는 전략이 중요하다”고 했다.  한국투자증권은 최근 미니스탁과 모이다를 분리해 내놓았다. 미니스탁은 해외주식을 소수점으로 나눠 1000원 단위로 거래하는 해외주식 투자를 지원한다. 모이다는 올해부터 본격화된 마이데이터를 바탕으로 투자 패턴 분석 등을 제공하는 생활 금융 플랫폼이다. 다양하게 앱을 나누고 다양한 투자 수요를 충족할 수 있는 서비스를 내놓겠다는 것이다. 최 본부장은 “인공지능(AI) 등의 기술을 

In [ ]:
ARTICLE = line_list[0][-1]
sentence_list = pro.sentence_splitter(ARTICLE)
for i, sentence in enumerate(sentence_list):
    print(i, sentence)

0 최서룡 한투증권 본부장 “오랜기간 얻은 데이터 바탕으로 더 정확한 투자 진단할 수 있고, 원앱보다 세분화된 앱이 중요” “디지털 전략의 핵심은 리스크 진단입니다.”  21일 동아일보와의 인터뷰에서 최서룡 한국투자증권 디지털플랫폼 본부장(사진)은 이같이 밝혔다.
1 최 본부장은 “유동성 장세가 지나며 얼마나 높은 수익률을 낼 수 있는지보다 얼마만큼 잃을 수 있는지 리스크를 계산하는 게 중요해졌다”고 했다.
2 최근 토스, 카카오 등 증권업에 진출하는 빅테크 기업들이 늘어나는 가운데 전통 증권사들도 디지털 역량 강화에 박차를 가하고 있다.
3 한국투자증권은 디지털플랫폼 본부를 두고 소수점 해외주식 거래 ‘미니스탁’, 마이데이터 ‘모이다’ 등 디지털플랫폼 구축과 운용에 적극 나서고 있다.
4 빅테크와 차별화된 경쟁력을 가지려면 기존 증권사만의 강점을 활용해야 한다는 게 최 본부장의 생각이다.
5 기존 증권사의 가장 큰 자산은 리스크 관리 노하우다.
6 그는 “오랜 기간 금융투자업계에서 얻은 데이터를 바탕으로 금융상품 투자 손실 가능성 및 비용 등을 기존 빅테크보다 정확하게 진단할 수 있다”고 했다.
7 빅테크의 장점으로 언급되는 ‘원앱’ 전략에 대해서도 무조건 따라 할 건 아니라고 보고 있다.
8 토스는 토스뱅크, 증권, 송금 서비스 등을 한데 모아놓은 ‘원앱’ 전략을 내놓았고, 간편함으로 시장의 호평을 받았다.
9 최 본부장은 “금융상품 투자부터 연금 투자, 선물 등 이미 많은 기능을 갖고 있어 이를 앱 하나로 모으는 것은 불가능하다. 오히려 기능을 세분화하고 기능에 맞게 앱을 추천하는 전략이 중요하다”고 했다.
10 한국투자증권은 최근 미니스탁과 모이다를 분리해 내놓았다.
11 미니스탁은 해외주식을 소수점으로 나눠 1000원 단위로 거래하는 해외주식 투자를 지원한다.
12 모이다는 올해부터 본격화된 마이데이터를 바탕으로 투자 패턴 분석 등을 제공하는 생활 금융 플랫폼이다.
13 다양하게 앱을 나누고 다양한 투자 수요를 충족할 수 있는 서비스를 내놓겠다는 것이

#### 2.2. 요약하기: TextRank

---

In [7]:
SUMMARIZER_TYPE = "textrank"
MAX_SENTENCES = 3

ds.summarize(SUMMARIZER_TYPE, MAX_SENTENCES, document="최서룡 한투증권 본부장은 이렇게 말했다. 최성욱 대표 디지털 서비스 강화 JT저축은행이 2015년 출범 이후 7년 만에 총자산 규모. ㄴㅇㄹㄴㅇㄹㄴㅇㄹㄴㅇㄹㄴㅇㄹㄴㅇㄹ")

NameError: ignored

In [ ]:
SUMMARIZER_TYPE = "textrank"
MAX_SENTENCES = 3

ds.summarize(SUMMARIZER_TYPE, MAX_SENTENCES, document=ARTICLE)

['최서룡 한투증권 본부장 “오랜기간 얻은 데이터 바탕으로 더 정확한 투자 진단할 수 있고, 원앱보다 세분화된 앱이 중요” “디지털 전략의 핵심은 리스크 진단입니다.”  21일 동아일보와의 인터뷰에서 최서룡 한국투자증권 디지털플랫폼 본부장(사진)은 이같이 밝혔다.',
 '한국투자증권은 디지털플랫폼 본부를 두고 소수점 해외주식 거래 ‘미니스탁’, 마이데이터 ‘모이다’ 등 디지털플랫폼 구축과 운용에 적극 나서고 있다.',
 '그는 “오랜 기간 금융투자업계에서 얻은 데이터를 바탕으로 금융상품 투자 손실 가능성 및 비용 등을 기존 빅테크보다 정확하게 진단할 수 있다”고 했다.']

### Assignment 07. 100개 기사 요약하기
> - 본 자료를 응용하여 여러개 기사를 추출요약하는 코드를 작성합니다.
> - 요약 알고리즘은 TextRank, 요약문은 최대 3개까지 생성합니다.
> - 기사 텍스트는 "data/article_sample.txt" 파일을 활용합니다.

---

In [ ]:
from teanaps.handler import FileHandler
from teanaps.nlp import Processing

fh = FileHandler()
pro = Processing()

PATH = "/content/article_토스뱅크_2022.02.22_2022.02.22_1.txt"
line_list = fh.load_txt(PATH)

SAVE_PATH = "/content/drive/MyDrive/advanced_text_mining/week_09/toss_result.xlsx"
f = open(SAVE_PATH, "w", encoding="utf-8")

for i, line in enumerate(line_list):
    ARTICLE = line[-1]    
    try :
        result_list = [] 
        target = ds.summarize(SUMMARIZER_TYPE, MAX_SENTENCES, document=ARTICLE)
        for x in target :
            result_list.append(x)    
    except : 
        continue

    for result in result_list:
        print(str(i+1) + "\t" + result)
        f.write(str(i+1) + "\t" + result + "\n")

f.close()

1	최서룡 한투증권 본부장 “오랜기간 얻은 데이터 바탕으로 더 정확한 투자 진단할 수 있고, 원앱보다 세분화된 앱이 중요” “디지털 전략의 핵심은 리스크 진단입니다.”  21일 동아일보와의 인터뷰에서 최서룡 한국투자증권 디지털플랫폼 본부장(사진)은 이같이 밝혔다.
1	한국투자증권은 디지털플랫폼 본부를 두고 소수점 해외주식 거래 ‘미니스탁’, 마이데이터 ‘모이다’ 등 디지털플랫폼 구축과 운용에 적극 나서고 있다.
1	그는 “오랜 기간 금융투자업계에서 얻은 데이터를 바탕으로 금융상품 투자 손실 가능성 및 비용 등을 기존 빅테크보다 정확하게 진단할 수 있다”고 했다.
2	전북銀, 암호화폐 거래소 손잡아 대구銀, 메타버스로 채용박람회 지방은행들이 ‘디지털실험’에 본격적으로 속도를 내고 있다.
2	비대면 상품, 모바일 앱 등 디지털서비스를 확대하는 것에서 나아가 암호화폐, 메타버스 등 블루오션에 적극적인 관심을 보이는 추세다.
2	21일 금융업계에 따르면 전북은행은 최근 가상자산(암호화폐) 거래소 ‘고팍스’와 실명확인 입출금 계정 발급 계약을 맺었다.
3	최성욱 대표 "디지털 서비스 강화" JT저축은행이 2015년 출범 이후 7년 만에 총자산 규모가 약 6배 늘어나 2조원을 돌파하면서 '대형 저축은행'에 한 걸음 가까워졌다.
3	22일 JT저축은행은 2015년 1월 SC저축은행 인수 시 약 3천369억 원이던 총자산이 지난 1월 말 기준 약 2조1천500억원으로 증가했다고 밝혔다.
3	최성욱 JT저축은행 대표는 "디지털 서비스의 편의성이 나날이 중요해지는 이때 서비스 개선 뿐 아니라 대내외적 리스크 관리 역량을 더욱 강화해 지속 가능한 성장을 이어갈 것"이라고 말했다.
4	“디지털 서비스 및 리스크 관리 역량 강화” JT저축은행이 출범 7년 만에 총 자산 2조원을 돌파했다.
4	사진은 경기도 성남시에 위치한 JT저축은행 본사.[JT저축은행 제공]   [헤럴드경제=이태형 기자]JT저축은행이 2015년 출범 이후 총 자산 규모가 약 6배 늘어나며 총자산이 2조원을 돌파했다

In [ ]:
### 점검
from teanaps.handler import FileHandler
from teanaps.nlp import Processing

fh = FileHandler()
pro = Processing()

In [ ]:
import pandas as pd
test = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/result.xlsx")
fh.load_txt('/content/article_토스뱅크_2022.02.22_2022.02.22_1.txt')

[['동아일보',
  '2022.02.22. 오전 3:03',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '1',
  '0',
  '0',
  '0',
  'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=020&aid=0003412466',
  '“증권사, 빅테크와 맞설 무기는 리스크 관리”',
  '최서룡 한투증권 본부장 “오랜기간 얻은 데이터 바탕으로 더 정확한 투자 진단할 수 있고, 원앱보다 세분화된 앱이 중요” “디지털 전략의 핵심은 리스크 진단입니다.”  21일 동아일보와의 인터뷰에서 최서룡 한국투자증권 디지털플랫폼 본부장(사진)은 이같이 밝혔다. 최 본부장은 “유동성 장세가 지나며 얼마나 높은 수익률을 낼 수 있는지보다 얼마만큼 잃을 수 있는지 리스크를 계산하는 게 중요해졌다”고 했다.  최근 토스, 카카오 등 증권업에 진출하는 빅테크 기업들이 늘어나는 가운데 전통 증권사들도 디지털 역량 강화에 박차를 가하고 있다. 한국투자증권은 디지털플랫폼 본부를 두고 소수점 해외주식 거래 ‘미니스탁’, 마이데이터 ‘모이다’ 등 디지털플랫폼 구축과 운용에 적극 나서고 있다.  빅테크와 차별화된 경쟁력을 가지려면 기존 증권사만의 강점을 활용해야 한다는 게 최 본부장의 생각이다. 기존 증권사의 가장 큰 자산은 리스크 관리 노하우다. 그는 “오랜 기간 금융투자업계에서 얻은 데이터를 바탕으로 금융상품 투자 손실 가능성 및 비용 등을 기존 빅테크보다 정확하게 진단할 수 있다”고 했다.  빅테크의 장점으로 언급되는 ‘원앱’ 전략에 대해서도 무조건 따라 할 건 아니라고 보고 있다. 토스는 토스뱅크, 증권, 송금 서비스 등을 한데 모아놓은 ‘원앱’ 전략을 내놓았고, 간편함으로 시장의 호평을 받았다. 최 본부장은 “금융상품 투자부터 연금 투자, 선물 등 이미 많은 기능을 갖고 있어 이를 앱 하나로 모으

In [ ]:
SAVE_PATH = "/content/drive/MyDrive/advanced_text_mining/week_09/toss_result.xlsx"
f = open(SAVE_PATH, "w", encoding="utf-8")

for i, line in enumerate(line_list):
    ARTICLE = line[-1]    
    try :
        result_list = [] 
        target = ds.summarize(SUMMARIZER_TYPE, MAX_SENTENCES, document=ARTICLE)
        for x in target :
            result_list.append(x)    
    except : 
        continue

    for result in result_list:
        print(str(i+1) + "\t" + result)
        f.write(str(i+1) + "\t" + result + "\n")

f.close()